In [14]:
from os.path import join
from os import mkdir
import pandas as pd
from tqdm import tqdm
import shutil
from src import utils


In [15]:
# Project path
project_dir = os.path.abspath('')[:-9]
# Load enviromental variables

env_var = utils.load_env_variables(project_dir)
env_var["root_path"] = "/home/tpinho/IJGIS/Datasets/Brazil_Election_2018"

In [16]:
dataset = "Original"
scv = "RBuffer"
fs_methods = [f for f in os.listdir(os.path.join(env_var["root_path"], dataset, "results", scv, "evaluations"))]
fs_methods.remove("LatLon")
fs_methods.remove("XCFS_Local_Meta_All_OLS_Error")
fs_methods = ["CFS", "CFS_Local_Mean_All", "CFS_Local_Mean_K7", "CFS_Local_Meta_All_OLS", "CFS_Local_Meta_K7_OLS"]
rename_col = {"CFS": "Global",
              "CFS_Local_Mean_All": "Local Mean K=All",
              "CFS_Local_Mean_K7": "Local Mean K=7",
              "CFS_Local_Meta_All_OLS": "Local Meta K=All",
              "CFS_Local_Meta_K7_OLS": "Local Meta K=7"}


In [17]:
ml_methods = ["KNN", "OLS", "Lasso", "Ridge", "ElasticNet", "DT", "LGBM", "MLP", "SVM"]
mean_rank = {"Methods": [],
             "Global": [],
             "Local Mean K=All": [],
             "Local Mean K=7": [],
             "Local Meta K=All": [],
             "Local Meta K=7": []
             }
mean_sp_rank = {"Methods": [],
             "Global": [],
             "Local Mean K=All": [],
             "Local Mean K=7": [],
             "Local Meta K=All": [],
             "Local Meta K=7": []
             }

mean_stde_rank = {"Methods": [],
             "Global": [],
             "Local Mean K=All": [],
             "Local Mean K=7": [],
             "Local Meta K=All": [],
             "Local Meta K=7": []
             }

mean_mse = {"Methods": [],
             "Global": [],
             "Local Mean K=All": [],
             "Local Mean K=7": [],
             "Local Meta K=All": [],
             "Local Meta K=7": []
             }
for ml_method in tqdm(ml_methods):
    results = pd.DataFrame()
    results_sp = pd.DataFrame()
    results_stde = pd.DataFrame()
    for fs_method in fs_methods:
        metrics = pd.read_csv(os.path.join(env_var["root_path"], dataset, "results", scv, "evaluations", fs_method, ml_method, "metrics_with_sp.csv"))
        metrics.set_index("FOLD", inplace=True)
        results[fs_method] = metrics["RMSE"]
        
        results_sp[fs_method] = metrics["SPEARMAN"]
        results_stde[fs_method] = metrics["STDE"]
    
    results.rename(rename_col, axis=1, inplace=True)
    results.to_csv(os.path.join(env_var["root_path"], dataset, "comparison_by_fold", f"{ml_method}_MSE.csv"))
    results_rank = results.rank(axis=1)
    results_rank.to_csv(os.path.join(env_var["root_path"], dataset, "comparison_by_fold", f"{ml_method}_RANK.csv"))
    
    results_sp.rename(rename_col, axis=1, inplace=True)
    results_sp.to_csv(os.path.join(env_var["root_path"], dataset, "comparison_by_fold", f"{ml_method}_SPEARMAN.csv"))
    results_sp_rank = results_sp.rank(axis=1, ascending=False)
    results_sp_rank.to_csv(os.path.join(env_var["root_path"], dataset, "comparison_by_fold", f"{ml_method}_SP_RANK.csv"))
    
    results_stde.rename(rename_col, axis=1, inplace=True)
    results_stde.to_csv(os.path.join(env_var["root_path"], dataset, "comparison_by_fold", f"{ml_method}_STDE.csv"))
    
    
    mean_mse["Methods"].append(ml_method)
    results.drop(index=[12, 14], inplace=True)
    for col in results.columns:
        mean_mse[col].append(results[col].mean())
    
    mean_rank["Methods"].append(ml_method)
    for col in results.columns:
        mean_rank[col].append(results_rank[col].mean())
        
    mean_sp_rank["Methods"].append(ml_method)
    for col in results_sp.columns:
        mean_sp_rank[col].append(results_sp_rank[col].mean())
    results_stde.drop(index=[12, 14], inplace=True)
    mean_stde_rank["Methods"].append(ml_method)
    for col in results_stde.columns:
        mean_stde_rank[col].append(results_stde[col].mean())

pd.DataFrame(mean_rank).to_csv(os.path.join(env_var["root_path"], dataset, "comparison", "RANK_BY_FOLDS.csv"), index=False)
pd.DataFrame(mean_sp_rank).to_csv(os.path.join(env_var["root_path"], dataset, "comparison", "SP_RANK_BY_FOLDS.csv"), index=False)
pd.DataFrame(mean_stde_rank).to_csv(os.path.join(env_var["root_path"], dataset, "comparison", "STDE.csv"), index=False)
pd.DataFrame(mean_mse).to_csv(os.path.join(env_var["root_path"], dataset, "comparison", "MSE.csv"), index=False)



100%|██████████| 9/9 [00:00<00:00, 27.28it/s]
